In [67]:
import os
import pandas as pd
import re
from datetime import datetime
from openpyxl import load_workbook


In [68]:
#extract the excel file the fund name with short name
file = pd.read_excel('Peer_List_New.xlsx')
file

,SecId,Number,FundStandardName,Short Name
0,NaN,1,Paradice Australian Mid Cap,Par
1,F0AUS05H5K,2,Aberdeen Ex-20 Australian Equities Fund,Abe
2,F0000118ZX,3,Antares Elite Opportunities Model,Eli
3,F000014995,4,Antares Ex-20 SMA Model,Ant
4,F00000XXND,5,BetaShares Aus Ex-20 Port Divrs ETF,Bet
5,F0AUS072J1,6,BT Challenger Select Aus Shr WFS,Cha
6,F00000Z3VZ,7,Chester High Conviction Fund,Che
7,F0AUS05KBX,8,Dimensional Australian Value Trust,Dim
8,F00000Q6LE,9,Fidelity Future Leaders Fund,Fid
9,F0AUS05DRS,10,Lazard Australian Equity Fund,Laz


In [73]:
#create a new dataframe with the file names
data = {'File Name':[]}
df = pd.DataFrame(data)
df['Company Name'] = ''

for filename in os.listdir('.'):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        df = df.append({'File Name': filename}, ignore_index=True)
        
#drop the peer list file        
df = df.drop(df.index[0])
df

C:\Users\HenryNguyen\AppData\Local\Temp\ipykernel_5648\356578053.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'File Name': filename}, ignore_index=True)
C:\Users\HenryNguyen\AppData\Local\Temp\ipykernel_5648\356578053.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'File Name': filename}, ignore_index=True)


,File Name,Company Name
1,The Montgomery Fund-Paradice Australian Mid Ca...,NaN


In [74]:
#go through the dataframe and extract the name of the fund into another column
for index, row in df.iterrows():
    match = re.search(r'^(.*)-Paradice', row['File Name'])
    if match:
        company_name = match.group(1).strip()
        row['Company Name'] = company_name
        

In [75]:
abbr_dict = file.set_index('FundStandardName')['Short Name'].to_dict()
#create a column for the abbreviation for the fund name
df['Abbr'] = df['Company Name'].map(abbr_dict)


In [76]:
#create new columns for the month and year
df['Date'] = df['File Name'].str.extract(r'AUD-(\d{8})')
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df['Month'] = df['Date'].dt.strftime('%m')
df['Year'] = df['Date'].dt.strftime('%y')
#create a new column for the new file name
df['New Name'] = df['Abbr'] + df['Month'] + df['Year'] + '.xlsx'
df

,File Name,Company Name,Abbr,Date,Month,Year,New Name
1,The Montgomery Fund-Paradice Australian Mid Ca...,The Montgomery Fund,Mon,2023-06-30,06,23,Mon0623.xlsx


In [77]:
#change the names of the file in the folder
for index, row in df.iterrows():
    old_name = row['File Name']
    new_name = row['New Name']
    
    old_file = os.path.join('.', old_name)
    new_file = os.path.join('.', new_name)
    
    os.rename(old_file, new_file)